In [3]:
import pandas as pd
d = {"headers": {"SurveyLog": ["id", "action", "question_id", "answer_id", "q_num", "timestamp"]},"rows": {"SurveyLog": [[5, "show", 285, None, 1, 123], [5, "answer", 285, 124124, 1, 124], [5, "show", 369, None, 2, 125], [5, "skip", 369, None, 2, 126]]}}
pd.DataFrame(d['rows']['SurveyLog'], columns=d['headers']['SurveyLog']).to_csv("./SurveyLog.txt", index=None)

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("578").config("pyspark.sql.shuffle.partition","4").getOrCreate()

In [5]:
s = spark.read.option("header", True).csv("./SurveyLog.txt")
s.show()

+---+------+-----------+---------+-----+---------+
| id|action|question_id|answer_id|q_num|timestamp|
+---+------+-----------+---------+-----+---------+
|  5|  show|        285|     null|    1|      123|
|  5|answer|        285| 124124.0|    1|      124|
|  5|  show|        369|     null|    2|      125|
|  5|  skip|        369|     null|    2|      126|
+---+------+-----------+---------+-----+---------+



The answer rate for a question is the number of times a user answered the question by the number of times a user showed the question.

Write an SQL query to report the question that has the highest answer rate. If multiple questions have the same maximum answer rate, report the question with the smallest question_id.

The query result format is in the following example.

In [23]:
from pyspark.sql.functions import col
answered = s.where("action = 'answer'").select("question_id").groupBy("question_id").count().withColumnRenamed("count","answer_count")
showed = s.where("action='show'").select("question_id").groupBy("question_id").count().join(answered, "question_id", "leftouter").select("question_id", (col("answer_count")/col("count")).alias("rate")).orderBy("rate", ascending=False)

In [24]:
showed.first()

Row(question_id='285', rate=1.0)